In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import numpy as np
import os
import faiss


In [2]:
# You API Key and intitialize mistral client
api_key = "YOUR API KEY"
#model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

In [72]:
# split the datset into chunks
# smaller length chunks do help while in longer length chunks information gets lost
# but we need maintain coherence between the chunks
# splitting based on indidvidual questions answer should help as well but that might not generalize well
# instead splitting based on headings sub headings, pargraph, but we have to keep in mind longer chunk sizes are not going to help 
# and too small might not capture the information
# balance needs to bemaintained with experimentation
f = open("dataset.txt", "r")
text = f.read()

chunk_size = 512
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
print(len(chunks))

33


In [73]:

def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
          model="mistral-embed",
          input=input
      )
    return embeddings_batch_response.data[0].embedding

In [5]:
#get mistral embeddings for similarity matching
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [75]:
#create a vector database for storing all the embeddings 
#we are using open source faise vector database, but others can be explored as well
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [7]:
question = "I had just logged on and saw that my tracker was expired. I didn't realize it expired as I hadn't seen any info on your web site regarding that. Can I get it going again or do I need to make a new tracker?"
# get embeddings for question
question_embeddings = np.array([get_text_embedding(question)])

In [35]:
# Seach for similar embeddings using nearest neighbors approach and distance as metric
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [9]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [20]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)


In [21]:
print(run_mistral(prompt))

Based on the provided context information, it seems that you can't renew an expired tracker. Instead, you would need to create a new one. However, the context doesn't explicitly state that expired trackers can't be renewed, so it would be best to reach out to the support team to confirm. They may be able to help you restore the expired tracker or guide you through creating a new one.

In summary, while it appears that you may need to create a new tracker, it is recommended to contact the support team for clarification and assistance.


In [50]:
question2 = "How to use Flexible dates feature for tracker" 
question2_embeddings = np.array([get_text_embedding(question2)])
D, I = index.search(question_embeddings, k=3) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt2 = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question2}
Answer:
"""
# 3 most similar embeddings helped rather than 2

In [51]:
print(run_mistral(prompt2))

To use the Flexible dates feature for a tracker, follow these steps:

1. Begin creating a tracker as you normally would.
2. When selecting dates, look for an option to check "Flexible +-1" or "+-2" days in small orange font.
3. This option will search for availability within 1 or 2 days before or after your specified date(s).
4. Make sure you have the Adventurer Mon subscription, as this feature is only available for that tier.
5. Check the box for the desired flexible date range.
6. Continue creating your tracker and save it. The tracker will now look for availability within the flexible date range you've chosen.


In [52]:
#Evaluation Technique
#We can feed the response from LLM into it again and ask whether it 
#think it has answered question correctly and if yes then with how much percentage
answer2 = f"""To use the Flexible dates feature for a tracker, follow these steps:

1. Begin creating a tracker as you normally would.
2. When selecting dates, look for an option to check "Flexible +-1" or "+-2" days in small orange font.
3. This option will search for availability within 1 or 2 days before or after your specified date(s).
4. Make sure you have the Adventurer Mon subscription, as this feature is only available for that tier.
5. Check the box for the desired flexible date range.
6. Continue creating your tracker and save it. The tracker will now look for availability within the flexible date range you've chosen."""

prompt_e = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information, do you think, you have answered your question correctly for following question and answer. 
Can you please answer yes or no with the percentage confidence you have in the given answer.
Query: {question2}
Answer: {answer2}
"""
# 3 most similar embeddings helped rather than 2

In [53]:
print(run_mistral(prompt_e))

Yes, 100%

The given answer correctly describes the steps to use the Flexible dates feature for a tracker, as outlined in the context information. The answer accurately mentions the orange font, the availability search within 1 or 2 days before or after the specified date(s), and the requirement of the Adventurer Mon subscription for the feature.


In [59]:
question3 = "Which number will I get call from when reservenature notifies about availability" 
question3_embeddings = np.array([get_text_embedding(question3)])
D, I = index.search(question3_embeddings, k=3) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt3 = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query. 
Please only answer the question if you have 100% confidence, other wise please say you don't know
Query: {question3}
Answer:
"""
#(wrong answer) --> You will get a call from +1 (669) 208-5607 when Reserve Nature notifies you about availability, with 95% confidence interval
#for this question may be adding this question in multiple forms in dataset.txt will help. 
#Seems like we this information is lost in longer chunk size. May be using smaller chunk size but then it will affect other questions 
#whose information is captured in longer chunk size


In [60]:
print(run_mistral(prompt3))

The contact number for Reserve Nature is +1 (669) 208-5607. Therefore, you will likely receive a call from this number when Reserve Nature notifies you about availability. However, I cannot say this with 100% certainty as the text does not explicitly state that this is the number they will use to call customers about availability. It is possible that they may use a different number for this purpose. If you want to confirm, you may want to contact Reserve Nature directly.


In [61]:
#Evaluation Technique (may be try a different LLM), and try to use the answer whichis average answer, if answers embeddings are close?
#We can feed the response from LLM into it again and ask whether it 
#think it has answered question correctly and if yes then with how much percentage
answer3 = f"""
You will get a call from +1 (669) 208-5607 when Reserve Nature notifies you about availability.
"""

prompt_e = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information, do you think, you have answered your question correctly for following question and answer. 
Can you please answer yes or no with the percentage confidence you have in the given answer.
Query: {question3}
Answer: {answer3}
"""

In [62]:
print(run_mistral(prompt_e))

Yes, 99% confident. The phone number +1 (669) 208-5607 is listed in the provided context information as a contact number for Reserve Nature.


In [67]:
question3 = "Will I get refund if I don't find availability for my campsite?" 
question3_embeddings = np.array([get_text_embedding(question3)])
D, I = index.search(question3_embeddings, k=3) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt3 = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query. 
Please only answer the question if you have 100% confidence, other wise please say you don't know
Query: {question3}
Answer:
"""
# point to be noted
# docs need to be updated if there's any change. offer was for limited time, but it's still on docs, so llm just uses that info
# update docs

In [65]:
print(run_mistral(prompt3))

Yes, according to the context information, if the service is unable to find a spot for your desired campsite, they will give 100% credits back (limited time offer).


In [76]:
question3 = "How to Cancel Subscription / Pause Subscription" 
question3_embeddings = np.array([get_text_embedding(question3)])
D, I = index.search(question3_embeddings, k=3) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt3 = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query. 
Please only answer the question if you have 100% confidence, other wise please say you don't know
Query: {question3}
Answer:
"""

In [77]:
print(run_mistral(prompt3))

To cancel your subscription, you can click on the top right menu and select "Dashboard". Then, select Subscriptions from the grid. You will see options to cancel your subscription. To pause your subscription, follow the same steps until you reach the Subscriptions page. There, you will see an option to pause your subscription instead of canceling it. Note that when you pause your subscription, you still have the subscription until the next billing date. After that, your account won't be debited for the duration of the pause, and you are free to resume the subscription anytime you want.
